In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import json
import numpy as np
from scripts.load_data import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

import optuna

In [3]:
configuration = load_configuration("../configuration/configuration.json")

paths = configuration["paths"]
data_student_numbers_f = pd.read_excel(paths["path_student_count_first-years"])
data_student_numbers_h = pd.read_excel(paths["path_student_count_higher-years"])
data_student_numbers_v = pd.read_excel(paths["path_student_volume"])

In [4]:
combined_data_student_numbers = data_student_numbers_f.merge(data_student_numbers_h, on=["Croho groepeernaam", "Herkomst", "Collegejaar"])
combined_data_student_numbers = combined_data_student_numbers.rename(columns={"Aantal_studenten_x": "Aantal_studenten_f", "Aantal_studenten_y": "Aantal_studenten_h"})
combined_data_student_numbers

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
0,2012,M Geneeskunde,NL,180,868
1,2013,M Geneeskunde,NL,173,892
2,2014,M Geneeskunde,NL,177,914
3,2015,M Geneeskunde,NL,21,1086
4,2016,M Geneeskunde,NL,148,984
...,...,...,...,...,...
2104,2016,M Leraar Voorbereidend Hoger Onderwijs in Gods...,NL,1,1
2105,2014,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,3,1
2106,2015,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,1,1
2107,2017,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,2,1


In [5]:
combined_data_student_numbers[combined_data_student_numbers["Croho groepeernaam"] == "B Sociologie"]

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
1776,2012,B Sociologie,NL,49,76
1777,2013,B Sociologie,NL,51,88
1778,2014,B Sociologie,NL,38,108
1779,2015,B Sociologie,NL,32,105
1780,2016,B Sociologie,NL,44,95
1781,2017,B Sociologie,NL,31,98
1782,2018,B Sociologie,NL,43,89
1783,2019,B Sociologie,NL,48,99
1784,2020,B Sociologie,NL,63,99
1785,2021,B Sociologie,NL,86,111


In [6]:
predict_year = 2023
programme = "B Sociologie"
origin = "NL"
examtype = "Bac"

In [7]:
data = combined_data_student_numbers
train = data[(data["Collegejaar"] < predict_year)]
# test = data[(data["Collegejaar"] == predict_year) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == origin)].drop(['Aantal_studenten_h'], axis=1)
test = data[(data["Collegejaar"] == predict_year)]

train = train.drop_duplicates()

In [8]:
X_train = train.drop(['Aantal_studenten_h'], axis=1)
y_train = train.pop('Aantal_studenten_h')

In [9]:
numeric_cols = ['Collegejaar', 'Aantal_studenten_f']
categorical_cols = ['Croho groepeernaam', 'Herkomst']

numeric_transformer = "passthrough"  # No transformation for numeric columns
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_cols),
        ('categorical', categorical_transformer, categorical_cols)
    ])

X_train = preprocessor.fit_transform(X_train)
preprocessed_test = preprocessor.transform(test)

In [10]:
print(preprocessed_test.shape)
print(len(test))

(184, 123)
184


In [13]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0, 1)
    model = XGBRegressor(learning_rate=learning_rate)

    model.fit(X_train, y_train)

    predictie = model.predict(preprocessed_test)

    total_mae = 0.0
    i = 0
    for _, row in test.iterrows():
        programme = row["Croho groepeernaam"]
        origin = row["Herkomst"]
        higher_year_row = data_student_numbers_h[(data_student_numbers_h["Croho groepeernaam"] == programme) &
                                                        (data_student_numbers_h["Herkomst"] == origin) &
                                                        (data_student_numbers_h["Collegejaar"] == predict_year)]

        mae = abs(predictie[i] - higher_year_row["Aantal_studenten"].iloc[0])

        total_mae += mae
        i += 1

    print(f"Final MAE: {total_mae / i}")
    return total_mae / i

In [14]:
study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2024-05-07 11:26:44,427] A new study created in memory with name: no-name-7b819094-9526-44ee-96de-5db26e5d9855
[I 2024-05-07 11:26:45,472] Trial 0 finished with value: 19.199328552121703 and parameters: {'learning_rate': 0.05904011884215821}. Best is trial 0 with value: 19.199328552121703.


Final MAE: 19.199328552121703


[I 2024-05-07 11:26:46,145] Trial 1 finished with value: 19.827080628146295 and parameters: {'learning_rate': 0.047636140252034}. Best is trial 0 with value: 19.199328552121703.


Final MAE: 19.827080628146295


[I 2024-05-07 11:26:46,513] Trial 2 finished with value: 17.71652816430382 and parameters: {'learning_rate': 0.30989148982567805}. Best is trial 2 with value: 17.71652816430382.


...

Final MAE: 16.764774119238492


[I 2024-05-07 11:28:22,181] Trial 198 finished with value: 17.27264589395212 and parameters: {'learning_rate': 0.5893663188165881}. Best is trial 53 with value: 15.367107813894425.


Final MAE: 17.27264589395212


[I 2024-05-07 11:28:22,710] Trial 199 finished with value: 16.833295379967794 and parameters: {'learning_rate': 0.5238388707172675}. Best is trial 53 with value: 15.367107813894425.


Final MAE: 16.833295379967794
